# Exercise 4: Various Kernels
Use the dataset dist.csv, which is a small constructed toy set, it is visualized in Figure 4. The data is
Gaussian mixture data, so there is a well-defined true decision bound. For x ∈ [-6, 10], this is approximated
by

```math
d(x) =
\begin{cases} 
0.5 \left( 18 - 2x - \sqrt{-724 + 256x - 16x^2} \right), & \text{if } x > 3.94 \\
0.071 \left( 174 - 22x - \sqrt{23123 - 6144x + 288x^2} \right), & \text{otherwise.}
\end{cases}
```

Hyperparameters to tune:
- Linear Kernel: C
- RBF Kernel: C, gamma
- Polynomial Kernel: C, d, gamma (optional)

## Part 1: Tune Hyperparameters
Tune the necessary hyperparameters by, for instance, grid search. In this exercise we are concerned with
the hyperparameters given in Table 1. Every hyperparameter should be tested for at least 3 values
but you are free to add more testings. There is a desginated validation set that can be used for the
validation of the hyperparameters dist_val.csv

Import and prepare data and define the function for grid search

In [3]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import pandas as pd

train_data = pd.read_csv('./../resources/datasets/dist.csv', header=None, delimiter=';')
val_data = pd.read_csv('./../resources/datasets/dist_val.csv', header=None, delimiter=';')

X_train, y_train = train_data.iloc[:, :-1], train_data.iloc[:, -1]
X_val, y_val = val_data.iloc[:, :-1], val_data.iloc[:, -1]

def grid_search(param_grid):
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1945)

    all_results = {}
    best_params = {}
    best_scores = {}
    best_models = {}

    for kernel_type, params in param_grid.items():
        svc = SVC()

        def validation_scorer(estimator, X, y):
            estimator.fit(X, y)
            val_predictions = estimator.predict(X_val)
            return f1_score(y_val, val_predictions)

        grid_search = GridSearchCV(svc, params, scoring=validation_scorer, cv=stratified_kfold, return_train_score=False)
        grid_search.fit(X_train, y_train)
        results_df = pd.DataFrame(grid_search.cv_results_)
        all_results[kernel_type] = results_df[['params', 'mean_test_score']]
        best_params[kernel_type] = grid_search.best_params_
        best_scores[kernel_type] = grid_search.best_score_
        best_models[kernel_type] = grid_search.best_estimator_

    for kernel_type, results in all_results.items():
        print(f"Results for {kernel_type} kernel:")
        print(results)

    print("Best hyperparameters for each kernel type:", best_params)
    print("Validation F1 scores for each kernel type:", best_scores)

    return best_models

Run grid search

In [ ]:
# C_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
# gamma_values = [0.001, 0.01, 0.1, 1]
# degree_values = [2, 3, 4]

C_values = [0.1, 1]
gamma_values = [0.01, 0.1]
degree_values = [2, 3]

param_grid = {
    'linear': {'kernel': ['linear'], 'C': C_values},
    'rbf': {'kernel': ['rbf'], 'C': C_values, 'gamma': gamma_values},
    'poly': {'kernel': ['poly'], 'C': C_values, 'degree': degree_values, 'gamma': gamma_values}
}

grid_search(param_grid)

### Best Hyperparameters

**Linear Kernel**
- **C**: 0.1
 
**RBF Kernel**
- **C**: 1
- **Gamma**: 0.1
 
**Polynomial Kernel**
- **C**: 1000
- **Gamma**: 1
- **Degree**: 3


Redefine parameter grid to search around the previously identified best values

In [4]:
refined_param_grid = {
    'linear': {
        'kernel': ['linear'],
        'C': [0.05, 0.075, 0.1, 0.125, 0.15]
    },
    'rbf': {
        'kernel': ['rbf'],
        'C': [0.5, 0.75, 1, 1.25, 1.5],
        'gamma': [0.05, 0.075, 0.1, 0.125, 0.15]
    },
    'poly': {
        'kernel': ['poly'],
        'C': [500, 750, 1000, 1250, 1500],
        'degree': [2, 3, 4], 
        'gamma': [0.5, 0.75, 1, 1.25, 1.5]
    }
}

models = grid_search(refined_param_grid)

## Part 2: Plots

For each kernel, produce a plot of the decision boundary for the best models together with the data.
If you want you can also include the true decision boundary as a comparison.